In [1]:
from elasticsearch import Elasticsearch

# Verbindung herstellen

In [2]:
es = Elasticsearch(["http://[::1]:9200"],
    basic_auth=("Student", "DieserVortragSollteMit_1_BewertetWerden!"),
 )
es.info()

ObjectApiResponse({'name': 'a521732bdff1', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Z_OTROEAS5KUKjzoOw_YTw', 'version': {'number': '8.6.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '2d58d0f136141f03239816a4e360a8d17b6d8f29', 'build_date': '2023-02-13T09:35:20.314882762Z', 'build_snapshot': False, 'lucene_version': '9.4.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
# Check Connection
if es.ping():
    print("Elasticsearch is connected!")
else:
    print("Elasticsearch is not connected!")

Elasticsearch is connected!


In [ ]:
# Überprüfen, ob Elasticsearch erreichbar ist
def check_connection():
    if es.ping():
        print("Elasticsearch is connected!")
    else:
        print("Elasticsearch is not connected!")

# Index erstellen
def create_index(index_name):
    es.indices.create(index=index_name)

# Index löschen
def delete_index(index_name):
    es.indices.delete(index=index_name)

# Dokument indizieren
def index_document(index_name, doc_id, document):
    es.index(index=index_name, id=doc_id, body=document)

# Dokument abrufen
def get_document(index_name, doc_id):
    return es.get(index=index_name, id=doc_id)

# Dokument aktualisieren
def update_document(index_name, doc_id, update_body):
    es.update(index=index_name, id=doc_id, body=update_body)

# Dokument löschen
def delete_document(index_name, doc_id):
    es.delete(index=index_name, id=doc_id)

# Einfache Suche
def search(index_name, query):
    return es.search(index=index_name, body={"query": {"match": query}})

# Beispielverwendung
if __name__ == "__main__":
    check_connection()
    index_name = "example_index"
    doc_id = 1
    document = {"name": "John Doe", "age": 30, "city": "New York"}

    create_index(index_name)
    index_document(index_name, doc_id, document)
    print("Indexed document:", get_document(index_name, doc_id))

    update_body = {"doc": {"age": 31}}
    update_document(index_name, doc_id, update_body)
    print("Updated document:", get_document(index_name, doc_id))

    query = {"name": "John Doe"}
    print("Search results:", search(index_name, query))

    delete_document(index_name, doc_id)
    delete_index(index_name)